# Web scraping to get the images for the artworks

In [8]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

pd.set_option('display.max_columns', None)

img_tags_progress = pd.read_csv('../art-sense/notebooks/img_tags_progress.csv')
img_tags_df = pd.read_csv('../art-sense/data/clean/img_tags_df.csv')


C:\Users\PC\AppData\Local\Temp\ipykernel_2700\975401515.py:9: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  img_tags_progress = pd.read_csv('../art-sense/notebooks/img_tags_progress.csv')


In [9]:
#img_tags_df['image_url'] = img_tags_progress['image_url']
display(img_tags_df)

,object_id,link_resource,tags,image_url
0,33,http://www.metmuseum.org/art/collection/search/33,Men|Abraham Lincoln|Portraits,https://collectionapi.metmuseum.org/api/collec...
1,34,http://www.metmuseum.org/art/collection/search/34,Landscapes|Boats,https://collectionapi.metmuseum.org/api/collec...
2,35,http://www.metmuseum.org/art/collection/search/35,Animals|Garlands|Birds|Men,https://collectionapi.metmuseum.org/api/collec...
3,36,http://www.metmuseum.org/art/collection/search/36,NaN,https://collectionapi.metmuseum.org/api/collec...
4,37,http://www.metmuseum.org/art/collection/search/37,Cannons|Swords|Men,https://collectionapi.metmuseum.org/api/collec...
...,...,...,...,...
48285,894449,http://www.metmuseum.org/art/collection/search...,NaN,NaN
48286,894685,http://www.metmuseum.org/art/collection/search...,NaN,NaN
48287,894734,http://www.metmuseum.org/art/collection/search...,NaN,NaN
48288,895183,http://www.metmuseum.org/art/collection/search...,NaN,NaN


In [10]:
last_row_with_image_url = img_tags_df[img_tags_df['image_url'].notna()].tail(1)
display(last_row_with_image_url)

,object_id,link_resource,tags,image_url
10326,24565,http://www.metmuseum.org/art/collection/search...,NaN,https://collectionapi.metmuseum.org/api/collec...


In [11]:
# Find the index of the row where object_id is 24575 (or row 19996)
index_split = img_tags_df[img_tags_df['object_id'] == 24565].index[0]

# Create the two DataFrames
df_before = img_tags_df.iloc[:index_split+1]  # Including the row with object_id 24575
df_after = img_tags_df.iloc[index_split+1:]  # Rows after the split

# Display the new DataFrames
display(df_before.shape)  # Show the last few rows of the first DataFrame
display(df_after.shape)  # Show the first few rows of the second DataFrame


(10327, 4)

(37963, 4)

In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

# Define the function to scrape image URLs
def fetch_image_url_from_web(link):
    """
    Fetches the image URL from a webpage given its link.
    """
    try:
        response = requests.get(link)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            img_tag = soup.find('img', {'id': 'artwork__image'})
            if img_tag and img_tag.get('src'):
                return img_tag['src']  # Return the 'src' of the image
        return None  # Return None if no image found
    except Exception as e:
        print(f"Error fetching image from {link}: {e}")
        return None

# Process the DataFrame and scrape the images
def scrape_images_from_df(df, link_col='link_resource', image_col='image_url', backup_interval=5000, backup_file="backup_amg_tags.csv"):
    """
    Processes the DataFrame to fetch image URLs for each link in the 'link_resource' column.
    Periodically saves progress to a backup file.
    """
    # Initialize the progress bar
    with tqdm(total=len(df), desc="Scraping Images") as pbar:
        for i, row in df.iterrows():
            if pd.isna(row[image_col]):  # Only process rows with missing image URLs
                image_url = fetch_image_url_from_web(row[link_col])
                df.at[i, image_col] = image_url  # Update the DataFrame
                
            # Save a backup after every `backup_interval` rows
            if (i + 1) % backup_interval == 0:
                df.to_csv(backup_file, index=False)
                print(f"Backup saved at row {i + 1} to {backup_file}")
            
            # Update progress bar
            pbar.update(1)

    # Final save
    df.to_csv(backup_file, index=False)
    print(f"Scraping completed. Final backup saved to {backup_file}.")
    return df

df_after = img_tags_df.iloc[index_split+1:]
# Scrape the images
updated_df = scrape_images_from_df(df_after, backup_interval=5000, backup_file="df_after_progress.csv")

# Display the result
print(updated_df)

Scraping Images:  12%|█▏        | 4673/37963 [1:24:21<9:55:07,  1.07s/it] 

Backup saved at row 15000 to df_after_progress.csv


Scraping Images:  25%|██▌       | 9673/37963 [2:50:06<8:46:00,  1.12s/it] 

Backup saved at row 20000 to df_after_progress.csv


Scraping Images:  39%|███▊      | 14673/37963 [4:15:33<6:23:54,  1.01it/s] 

...

Backup saved at row 40000 to df_after_progress.csv


Scraping Images:  91%|█████████▏| 34673/37963 [9:53:13<52:31,  1.04it/s]  

Backup saved at row 45000 to df_after_progress.csv


Scraping Images: 100%|██████████| 37963/37963 [10:50:04<00:00,  1.03s/it]  

Scraping completed. Final backup saved to df_after_progress.csv.
       object_id                                      link_resource tags  \
10327      24615  http://www.metmuseum.org/art/collection/search...  NaN   
10328      24616  http://www.metmuseum.org/art/collection/search...  NaN   
10329      24617  http://www.metmuseum.org/art/collection/search...  NaN   
10330      24618  http://www.metmuseum.org/art/collection/search...  NaN   
10331      24619  http://www.metmuseum.org/art/collection/search...  NaN   
...          ...                                                ...  ...   
48285     894449  http://www.metmuseum.org/art/collection/search...  NaN   
48286     894685  http://www.metmuseum.org/art/collection/search...  NaN   
48287     894734  http://www.metmuseum.org/art/collection/search...  NaN   
48288     895183  http://www.metmuseum.org/art/collection/search...  NaN   
48289     896897  http://www.metmuseum.org/art/collection/search...  NaN   

                      

In [13]:
df_after_progress = pd.read_csv('df_after_progress.csv')
merged_df = pd.concat([df_before, df_after_progress], ignore_index=True)

In [19]:
merged_df.to_csv('../art-sense/data/clean/images_and_tags.csv')